In [1]:
import sys
sys.path.append('c:\\Users\\rokas\\Documents\\GitHub\\BCI\\mi-bci\\code')

# Import ml_pipelines from the file you saved earlier
from pipelines.ml_pipelines import all_ml_pipelines
from evaluation import train_model

from helper_functions import setup_logger, load_procesed_data
from helper_functions import process_mi_epochs
from datasets import Lee2019
import os
import mne


log = setup_logger("Lee_pseudo_online", log_file=os.path.join("logs","Lee_pseudo_online.log"))
dataset = Lee2019()
dataset_no = 20
paradigm = "MI"
subject = 2
run = 1
data = load_procesed_data(dataset_no, paradigm, subject, run, include=['epochs_raw'])
#data = load_procesed_data(dataset_no, paradigm, subject, run, include=['epochs_raw', 'epochs_raw_autoreject'])
epochs = data["epochs_raw"]
epochs_p = process_mi_epochs(epochs)
pipeline = all_ml_pipelines['CSP_SVM']

raw = dataset.load_one_raw_fif(subject=subject,paradigm=paradigm,run=run)
# raw_raw = raw.copy()# kaži ar reikia
# event_id = dict(left_hand=2, right_hand=1)
# events_raw = mne.find_events(raw)

raw = process_mi_epochs(raw)
trained_model = train_model(data=epochs_p, pipeline=pipeline,log=log, data_type='epochs')

Reading c:\Users\rokas\Documents\GitHub\BCI\mi-bci\data\procesed\20\MI\2\1\s02.01_epochs_raw-epo.fif ...
    Found the data of interest:
        t =   -1200.00 ...    5200.00 ms
        0 CTF compensation matrices available
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Setting up band-pass filter from 8 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 8.00, 30.00 Hz: -6.02, -6.02 dB

Opening raw data file c:\Users\rokas\Documents\GitHub\BCI\mi-bci\data\raw_fif\20\MI\s02.01_raw.fif...
    Range : 0 ... 1434799 =      0.000 ...  1434.799 secs
Ready.
Reading 0 ... 1434799  =      0.000 ...  1434.799 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw 

2025-01-08 14:04:16,961 - Lee_pseudo_online - INFO - Preparing data for training.
2025-01-08 14:04:17,007 - Lee_pseudo_online - INFO - Training the model.


Computing rank from data with rank=None
    Using tolerance 3.1e-05 (2.2e-16 eps * 20 dim * 6.9e+09  max singular value)
    Estimated rank (data): 20
    data: rank 20 computed from 20 data channels with 0 projectors
Reducing data rank from 20 -> 20
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.


In [14]:
# huine iš GPT, rašyt pačiam
# galimai reiks dvigubo klasifikavimo

In [12]:
import numpy as np



def sliding_window_classification(raw, trained_model, confidence_threshold=0.8, window_size=2.5, step_size=0.1, log=None):
    """
    Perform sliding window classification on raw EEG data with a confidence threshold.

    Parameters:
    - raw: MNE Raw object containing continuous EEG data.
    - trained_model: Pre-trained pipeline or model (trained on 2.5s epochs).
    - confidence_threshold: Minimum confidence (e.g., 0.8) required to make a classification.
    - window_size: Size of the sliding window in seconds (default: 2.5s).
    - step_size: Step size for the sliding window in seconds (default: 0.1s).
    - log: Logger instance for logging information (optional).

    Returns:
    - classifications: List of classifications with timestamps, predicted classes, confidence levels, and scores.
    """
    if log is None:
        import logging
        logging.basicConfig(level=logging.INFO)
        log = logging.getLogger("sliding_window")

    sfreq = raw.info['sfreq']  # Sampling frequency
    window_samples = int(window_size * sfreq)
    step_samples = int(step_size * sfreq)

    data = raw.get_data()
    classifications = []

    log.info("Starting sliding window classification with %.2fs window size.", window_size)

    for start in range(0, data.shape[1] - window_samples + 1, step_samples):
        end = start + window_samples
        window_data = data[:, start:end]

        # Reshape window_data to match the expected format (e.g., n_samples, n_channels, n_times)
        window_data = window_data[np.newaxis, :, :]

        # Apply transformations if the model is a pipeline
        if hasattr(trained_model, 'transform'):
            try:
                window_data = trained_model.transform(window_data)
            except Exception as e:
                log.warning(f"Transformation failed for window {start}-{end}: {e}")
                continue

        # Calculate predictions, confidence, and score
        if hasattr(trained_model, 'predict_proba'):
            probabilities = trained_model.predict_proba(window_data)[0]
            predicted_class = np.argmax(probabilities)
            confidence = probabilities[predicted_class]
        elif hasattr(trained_model, 'decision_function'):
            decision_values = trained_model.decision_function(window_data)
            confidence = np.max(np.abs(decision_values))
            predicted_class = np.argmax(decision_values)
        else:
            predicted_class = trained_model.predict(window_data)[0]
            confidence = 1.0  # Assume confidence is 100% if not supported

        # Compute accuracy score if the model supports scoring
        score = None
        if hasattr(trained_model, 'score'):
            try:
                dummy_label = [predicted_class]
                score = trained_model.score(window_data, dummy_label)
            except Exception as e:
                log.warning(f"Failed to compute score for window {start}-{end}: {e}")

        # Apply the confidence threshold
        if confidence >= confidence_threshold:
            timestamp = (start + window_samples / 2) / sfreq
            classifications.append({
                'timestamp': timestamp,
                'class': predicted_class,
                'confidence': confidence,
                'score': score
            })
            log.info(f"Classification at {timestamp:.2f}s: Class {predicted_class} (Confidence: {confidence:.2f}, Score: {score})")

    log.info("Sliding window classification complete.")
    return classifications


In [13]:
classifications = sliding_window_classification(raw=raw,
                                                trained_model=trained_model, 
                                                confidence_threshold=0.8, 
                                                window_size=2.5,
                                                step_size=0.1,
                                                log=log)

2025-01-08 14:16:58,466 - Lee_pseudo_online - INFO - Starting sliding window classification with 2.50s window size.
2025-01-08 14:16:58,469 - Lee_pseudo_online - INFO - Classification at 1.25s: Class 0 (Confidence: 0.86, Score: 0.0)
2025-01-08 14:16:58,469 - Lee_pseudo_online - INFO - Classification at 1.35s: Class 0 (Confidence: 0.98, Score: 0.0)
2025-01-08 14:16:58,469 - Lee_pseudo_online - INFO - Classification at 1.45s: Class 0 (Confidence: 0.94, Score: 0.0)
2025-01-08 14:16:58,474 - Lee_pseudo_online - INFO - Classification at 1.55s: Class 0 (Confidence: 1.23, Score: 0.0)
2025-01-08 14:16:58,476 - Lee_pseudo_online - INFO - Classification at 1.65s: Class 0 (Confidence: 1.31, Score: 0.0)
2025-01-08 14:16:58,477 - Lee_pseudo_online - INFO - Classification at 1.75s: Class 0 (Confidence: 1.14, Score: 0.0)
2025-01-08 14:16:58,480 - Lee_pseudo_online - INFO - Classification at 1.85s: Class 0 (Confidence: 1.17, Score: 0.0)
2025-01-08 14:16:58,483 - Lee_pseudo_online - INFO - Classificati

In [5]:
classifications

[{'timestamp': 1.25, 'class': 0, 'confidence': 0.8603577554435522},
 {'timestamp': 1.35, 'class': 0, 'confidence': 0.9764784998961168},
 {'timestamp': 1.45, 'class': 0, 'confidence': 0.9356434891725359},
 {'timestamp': 1.55, 'class': 0, 'confidence': 1.2287756183813074},
 {'timestamp': 1.65, 'class': 0, 'confidence': 1.3097582687307816},
 {'timestamp': 1.75, 'class': 0, 'confidence': 1.1446212789922938},
 {'timestamp': 1.85, 'class': 0, 'confidence': 1.1707771745318905},
 {'timestamp': 1.95, 'class': 0, 'confidence': 1.2010761635651779},
 {'timestamp': 2.05, 'class': 0, 'confidence': 1.1357573785552488},
 {'timestamp': 2.15, 'class': 0, 'confidence': 1.1315149476203197},
 {'timestamp': 2.25, 'class': 0, 'confidence': 1.0925799863601138},
 {'timestamp': 15.05, 'class': 0, 'confidence': 0.9624038858896812},
 {'timestamp': 15.15, 'class': 0, 'confidence': 0.8009311203124787},
 {'timestamp': 15.25, 'class': 0, 'confidence': 1.1820209995293833},
 {'timestamp': 15.35, 'class': 0, 'confidence